# <font color=cyan> Import ERP Payment </font>

## Token generation and API Management

In [ ]:
#@title Import modules

import pandas as pd
import json
import requests, base64
import re
import pandas as pd
from io import StringIO
import csv
import copy

In [ ]:
#@title API Management

def login(client_id, client_secret):
    """
    :param client_id: str
    :param client_secret: str
    :return: encoded user name (input of the function get_token)
    """
    userpass = client_id + ':' + client_secret
    encoded_u = base64.b64encode(userpass.encode()).decode()
    return encoded_u

def get_token(encoded_u):
    url = "https://demo.kyriba.com/gateway/oauth/token"
    payload='grant_type=client_credentials&scope='
    headers = {
      'Authorization': 'Basic '+ encoded_u,
      'Content-Type': 'application/x-www-form-urlencoded'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    return json.loads(response.text)['access_token']

def input_to_api(txt,token,filename):
    """
    :param txt: str to try if it corresponds to the desired output
    :param token: str output of the get_token function
    :param filename: str should be unique each time to find the response of the query launched
    :return: fileId str
    """
    url = "https://demo.kyriba.com/gateway/api/v1/data?fileName="+filename
    payload = txt
    headers = {
      'Content-Type': 'text/plain;charset=utf-8',
      'Authorization': 'Bearer '+token
        }
    response = requests.request("POST", url, headers=headers, data=payload)
    return json.loads(response.text)['fileId']

def api_to_pt(fileid,templateref,token):
    url = "https://demo.kyriba.com/gateway/api/v1/process-templates/"+templateref+"/run?fileIds="+fileid
    payload={}
    headers = {'Authorization': 'Bearer '+token}
    response = requests.request("POST", url, headers=headers, data=payload)
    return json.loads(response.text)[0]['taskId']

def get_status(taskid,token):
    url = "https://demo.kyriba.com/gateway/api/v1/process-templates/"+taskid+"/status"
    payload={}
    headers = {'Authorization': 'Bearer '+token}
    response = requests.request("GET", url, headers=headers, data=payload)
    return json.loads(response.text)

def get_details(taskid,token):
    url = "https://demo.kyriba.com/gateway/api/v1/process-templates/"+taskid+"/details"
    payload={}
    headers = {'Authorization': 'Bearer '+token}
    response = requests.request("GET", url, headers=headers, data=payload)
    return json.loads(response.text)

def get_result_file(taskid, ref, token):
    url = "https://demo.kyriba.com/gateway/api/v1/process-templates/"+ref+"/files?taskId="+taskid
    payload={}
    headers = {'Authorization': 'Bearer '+token}
    response = requests.request("GET", url, headers=headers, data=payload)
    return response.text

In [ ]:
#@title Code for Importing through Process Templates

def test(client_id,client_secret, txt, filename):
    """
    :param client_id: str that we give you
    :param client_secret: str that we give you
    :param txt: str to try if it corresponds to the desired output
    :param filename: tr should be unique each time to find the response of the query launched
    :return True, None if the txt is in the correct format and no errors were detected
    else: return False, dataframe describing the error found
    This function returns a single error despite the fact that several errors may exist in the input.
    You can correct the error found and re-run the function to find the other errors if they exist.
    """
    template_refs = ['ERP_PY_API','PY_REJ_API']
    encoded_u = login(client_id,client_secret)
    token = get_token(encoded_u)
    fileid = input_to_api(txt,token,filename)
    taskid1 = api_to_pt(fileid,template_refs[0],token)
    if get_status(taskid1,token)['status']=='Complete':
        return True, None
    taskid2 = api_to_pt(fileid,template_refs[1],token)
    out = get_result_file(taskid2,template_refs[1],token)
    list_ = out.split('\n')
    rex = re.compile('["]')
    l = []
    for element in list_:
        l.append(rex.sub('',element).split(','))
    df = pd.DataFrame(l,columns = l[2])
    return False, df[df['File name']==filename+'_rej']

## Example

Here we have two errors transacation date=20211132 and currency_code = US8

In [ ]:
#@title Define file content and name

txt = "TR||37276137836630a8473b|US BF1030134 0200154009450|US8|2000.00|20211132|||Computers by Jenni||||1515 Main St.||Middlefield|94482|CA|||||||||||||0910000193||||||||||||||||||||||||\r\nIN|0|155|155|20211109|20211130|0.00|2000.00|||||"
filename='test_hackathon1'

# Code for authentication through CSV file
cfg = pd.read_csv("/content/config.csv")
client_id = cfg['client_id'].values[0]
client_secret = cfg['client_secret'].values[0]

In [ ]:
#@title Get imported files

v, result = test(client_id,client_secret,txt,filename)
result

In [ ]:
#@title Change content, reimport and get results
txt = "TR||37276137836630a8473b|US BF1030134 0200154009450|US8|2000.00|20211130|||Computers by Jenni||||1515 Main St.||Middlefield|94482|CA|||||||||||||0910000193||||||||||||||||||||||||\r\nIN|0|155|155|20211109|20211130|0.00|2000.00|||||"
v, result = test(client_id,client_secret,txt,filename)
result